用pytorch模块实现一个更简洁的lr，对应[3.3节](https://tangshusen.me/Dive-into-DL-PyTorch/#/chapter03_DL-basics/3.3_linear-regression-pytorch)

Tips:  
torch.utils.data模块提供了有关数据处理的工具；  
torch.nn模块定义了大量神经网络的层;  
torch.nn.init模块定义了各种初始化方法;  
torch.optim模块提供了很多常用的优化算法。

In [1]:
import torch
import numpy as np

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [3]:
import torch.utils.data as Data

In [4]:
# 用pytorch的data模块 快速定义一个数据迭代器
batch_size = 10
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)  

In [5]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 1.1580,  0.2000],
        [ 1.0561, -1.2385],
        [ 0.8146, -0.2831],
        [-0.8633, -1.2957],
        [ 0.1307, -1.2041],
        [ 0.8090,  0.2308],
        [ 0.1174, -0.8255],
        [-0.2861, -1.2815],
        [ 0.6644,  0.4466],
        [-0.0986,  0.5112]]) tensor([ 5.8506, 10.5214,  6.7865,  6.8848,  8.5550,  5.0354,  7.2439,  7.9858,
         4.0017,  2.2545])


In [6]:
import torch.nn as nn

In [7]:
# pytorch搭建模型的常用范式：nn.Module
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1) # 定义网络层
    def forward(self, x):  # 定义正向传播
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net)  # 模型参数存放在： net.linear.weight, net.linear.bias, 或 net.parameters()

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [8]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.2592,  0.0557]], requires_grad=True)
Parameter containing:
tensor([0.1954], requires_grad=True)


torch.nn仅支持输入一个batch的样本，不支持单个样本输入，如果只有单个样本，可使用input.unsqueeze(0)来添加一维。

In [9]:
from torch.nn import init

In [10]:
# 用init模块初始化模型参数
init.normal_(net.linear.weight, mean=0, std=0.01)  
init.constant_(net.linear.bias, val=0)

Parameter containing:
tensor([0.], requires_grad=True)

In [11]:
loss = nn.MSELoss()

In [12]:
import torch.optim as optim

In [13]:
optimizer = optim.SGD(net.parameters(), lr=0.03)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [14]:
num_epochs = 3
for epoch in range(1, num_epochs+1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print("epoch %d, loss: %f" % (epoch, l.item()))

epoch 1, loss: 0.000283
epoch 2, loss: 0.000057
epoch 3, loss: 0.000119


In [15]:
print(true_w, net.linear.weight)
print(true_b, net.linear.bias)

[2, -3.4] Parameter containing:
tensor([[ 2.0000, -3.3999]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1994], requires_grad=True)


待拓展-本小节还提到了：  
1. 用nn.Sequential搭建网络 
2. 设置非固定学习率
3. torch.optim模块中的其他优化算法：SGD、Adam和RMSProp等